In [1]:
import gensim.downloader as api
import gensim
from gensim.models import Phrases
from gensim.models import KeyedVectors, Word2Vec
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import ast
import json

In [2]:
filename = r'/home/miboj/NLP/document-summarizer/data/processed/articles.json'
file = open(filename, encoding='ascii', errors='ignore')
text = file.read()
file.close()

d = ast.literal_eval(text)

In [3]:
with open(filename) as json_file:
    data = json.load(json_file)

filename = r'/home/miboj/NLP/document-summarizer/data/raw/articles.json'
file = open(filename, encoding='ascii', errors='ignore')
text = file.read()
file.close()

json_content = ast.literal_eval(text)
samples = json_content[0:10]

In [4]:
print(samples[1])

{'content': ['The Japan Air Self-Defense Force (JASDF) has inducted its first Kawasaki RC-2 ELINT platform (left) into service at Iruma AB in western Tokyo. The type will replace the older NAMC YS-11EB (right) in the role. ', 'pic.twitter.com/0cdb213Phq', ' MELBOURNE, Australia  The Japan Air Self-Defense Force has inducted the first of a new intelligence-gathering aircraft into service, following a two-year flight test program.', ' The JASDF announced the induction of the RC-2 electronic intelligence, or elint, gathering aircraft at a ceremony held at Iruma Air Base in the western suburbs of the Japanese capital Tokyo on Oct. 1. ', " The RC-2 is based on Kawasaki Heavy Industries' C-2 airlifter and has been heavily modified with multiple aircraft fairings that contain antennas for detecting, receiving and classifying electronic emissions. The aircraft made its maiden flight in early 2018, though the variant had been in development since at least 2015.", ' Since that time, it underwent

In [5]:
tokens_list = []
for i in d:
    for sen in i['content']:
        tokens_list.append(sen)

In [6]:
import time
start_time = time.time()
sentences = []
word_count = 0
stpwrds = stopwords.words('english') + list(string.punctuation) + ['—', '“', '”', "'", "’"]
for e, i in enumerate(tokens_list):
    words = []
    a = nltk.word_tokenize(i)
    for word in a:
        if word not in stpwrds:
            words.append(word)
            word_count += 1
    sentences.append(words)
print("--- %s seconds ---" % (time.time() - start_time))

--- 32.34333944320679 seconds ---


In [7]:
print(len(sentences))
print(word_count)

111137
1782730


In [8]:
"""
sg - Training algorithm: 1 for skip-gram, 0 for CBOW
hs - If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
"""
model = Word2Vec(sentences, size=100, window=5, workers=12, sg=1, hs=1, compute_loss=True)

In [9]:
model.most_similar('weapon')

<ipython-input-9-ddf5759f98d5>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('weapon')


[('weapons', 0.7701378464698792),
 ('ground-to-ground', 0.7409212589263916),
 ('man-portable', 0.7301902770996094),
 ('Phalanx', 0.7298985123634338),
 ('anti-torpedo', 0.7249369621276855),
 ('day/night', 0.7246792316436768),
 ('close-in', 0.7213224172592163),
 ('designators', 0.7162144184112549),
 ('canister', 0.7033758163452148),
 ('shorter-range', 0.7002357244491577)]

In [10]:
model = model.wv

In [11]:
import re
def remove_empty_string(input_string):
    for e, i in enumerate(input_string):
        try:
            if i[-1] == ' ' and input_string[e+1][-1] == ' ':
                input_string[e] = i.rstrip()
        except IndexError:
            print('Out of index')
    joined_string = ''.join(input_string)
    for e, i in enumerate(joined_string):
        if i == ' ' and joined_string[e+1] == ' ':
            del i
    sentences = nltk.sent_tokenize(joined_string)
    return sentences


In [12]:
raw_string = [" ROME — Defying reports that their planned partnership is ", "doomed to fail", ", France’s Naval Group and ", "Italy’s Fincantieri", " have announced a joint venture to build and export naval vessels. ", " The two ", "state-controlled shipyards", " said they were forming a 50-50 joint venture after months of talks to integrate their activities. The move comes as Europe’s fractured shipbuilding industry faces stiffer global competition. ", " The firms said in a statement that the deal would allow them to “jointly prepare winning offers for binational programs and export market,” as well as create joint supply chains, research and testing. ", " Naval Group and Fincantieri first announced talks on cooperation last year after the latter negotiated a controlling share in French shipyard STX. But the deal was reportedly losing momentum due to resistance from French industry and a political row between France and Italy over migrants. ", " The new deal falls short of the 10 percent share swap predicted by French Economy and Finance Minister Bruno Le Maire earlier this year, and far short of the total integration envisaged by Fincantieri CEO Giuseppe Bono. ", " The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.”", " Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.”", " The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.", "Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.", "By giving us your email, you are opting in to the Early Bird Brief.", " When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.”", " However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design. ", "Competition flares up for the follow-on portion of a deal previously won by the French shipbuilder.", " The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies. The work would include providing a common combat management system. ", " The statement was cautious about future acceleration toward integration. “A Government-to-Government Agreement would be needed to ensure the protection of sovereign assets, a fluid collaboration between the French and Italian teams and encourage further coherence of the National assistance programs, which provide a framework and support export sales,” the statement said.", " But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.”", " ", "Sebastian Sprenger", " in Paris contributed to this report."]
sentences = remove_empty_string(raw_string)

In [13]:
# The 'skipthoughts' module can be found at the root of the GitHub  repository linked above
#import skipthoughts

# You would need to download pre-trained models first
#model = skipthoughts.load_model()

#encoder = skipthoughts.Encoder(model)
#encoded =  encoder.encode(sentences)
a = model['ROME']
a.shape

(100,)

In [14]:
def get_embedding(sentences):
    embeddings = []
    stpwrds = stopwords.words('english') + list(string.punctuation) + ['—', '“', '”', "'", "’"]

    for i in sentences:
        temp = []
        words = nltk.word_tokenize(i)
        for word in words:
            true_len = len(words) - len([w for w in stpwrds if w in words])
            #if word not in stpwrds:
            if word in model.vocab:
                v = model[word]
            #else:
            #    v = np.zeros(300,)
            temp.append(v)
            a = sum(temp)/true_len
            np_temp = np.array(a)
        #embeddings.append(temp)
        embeddings.append(np_temp)

    sentence_vectors = np.array(embeddings)
    return sentence_vectors

In [15]:
def get_sim_matrix(sentences, sentence_vectors):
    sim_mat = np.zeros([len(sentences), len(sentences)])
    sim_mat.shape  
    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    return sim_mat

In [16]:
def get_pagerank(sim_mat):
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    return scores

In [17]:
def get_summary(num_sentences, scores, sentences):
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    #num_of_sentences = 4
    summary = ''
    for i in range(num_sentences):
        summary += ranked_sentences[i][1]
        summary += " "
    return summary

In [18]:
a = """The statement called the joint venture the “first steps” toward the creation of an alliance that would create “a more efficient and competitive European shipbuilding industry.” Naval Group CEO Hervé Guillou, speaking at the Euronaval trade expo in Paris on Oct. 24, said the alliance is based on “two countries sharing a veritable naval ambition.” The joint venture is necessary because the “context of the global market has changed drastically,” he added, specifically mentioning new market entrants Russia, China, Singapore, Ukraine, India and Turkey.Sign up for the Early Bird Brief, the defense industry's most comprehensive news and information, straight to your inbox.By giving us your email, you are opting in to the Early Bird Brief.
But the firms were optimistic the deal would be “a great opportunity for both groups and their eco-systems, by enhancing their ability to better serve the Italian and French navies, to capture new export contracts, to increase research funding and, ultimately, improve the competitiveness of both French and Italian naval sectors.” Sebastian Sprenger in Paris contributed to this report.
When asked about an initial product to be tackled under the alliance, Guillou acknowledged: “The answer is simple: there is nothing yet.” However, the firms said they are working toward a deal to build four logistics support ships for the French Navy, which will be based on an Italian design.
The firms also plan to jointly bid next year on work for midlife upgrades for Horizon frigates, which were built by France and Italy and are in service with both navies."""

In [19]:
summary_samples = []
summary_len = []
for i in samples:
    i = remove_empty_string(i['content'])
    embeddings = get_embedding(i)
    sim_mat = get_sim_matrix(i, embeddings)
    scores = get_pagerank(sim_mat)
    sentence_length = len(i)
    summary = get_summary(int(sentence_length*0.3), scores, i)
    summary_samples.append(summary)
    summary_len.append(int(sentence_length*0.3))

In [20]:
sorted_summaries = []
for e, i in enumerate(summary_samples):
    a = nltk.sent_tokenize(i)
    o = samples[e]['content']
    b = remove_empty_string(o)
    #print(a)
    #print(b)
    res = [sort for x in b for sort in a if sort == x]
    sorted_summaries.append(res)

Out of index


In [21]:
for e, i in enumerate(sorted_summaries):
    print(e, ": ")
    print("len original: ", len(remove_empty_string(samples[e]['content'])))
    print("Summary len: ", summary_len[e])
    summary = ""
    for sen in i:
        summary += sen
        summary += " "
    print(summary)

0 : 
len original:  17
Summary len:  5
Allvin will succeed the current vice chief, Gen. Seve Wilson, who is expected to retire after 39 years in uniform. Wilson has served in his role since July 2016 and is the longest-serving vice chief in Air Force history. Gen. David Allvin is an experienced pilot who has commanded a n umber of wings, including the 438th Air Expeditionary Wing in Afghanistan. Allvin also commanded the 438th Air Expeditionary Wing in Kabul, Afghanistan, in 2010 and 2011, during which time he also served as the commanding general of NATO Air Training Command. Allvin also commanded the 618th Air and Space Operations Center at Scott Air Force Base, Illinois, and served as director of strategy, plans and policy at U.S. European Commands headquarters at Stuttgart-Vaihingen, Germany. 
1 : 
len original:  19
Summary len:  5
The Japan Air Self-Defense Force (JASDF) has inducted its first Kawasaki RC-2 ELINT platform (left) into service at Iruma AB in western Tokyo. pic.twitt

In [27]:
for e, i in enumerate(samples):
    print(e)
    sample = ""
    temp = remove_empty_string(i['content'])
    for t in temp:
        sample += t
        sample += " "
    print(sample)

0
 Lt. Gen. David Allvin was confirmed by the Senate to be the Air Forces next vice chief of staff in a late-night vote Wednesday. Allvins nomination to become vice chief and receive his fourth star was approved unanimously. In a Thursday release, Chief of Staff Gen. Charles CQ Brown applauded Allvins confirmation. I could not be more pleased with the confirmation of Gen. Allvin as the new vice chief of staff, Brown said. When it comes to leading at the highest levels of joint strategy and policy, and as someone who sets the standard for critical collaboration with our allies and partners, there is no one more qualified for the role of vice chief. Allvin will succeed the current vice chief, Gen. Seve Wilson, who is expected to retire after 39 years in uniform. Wilson has served in his role since July 2016 and is the longest-serving vice chief in Air Force history. Allvin now serves director of strategy, plans and policy for the Joint Staff at the Pentagon, and is a senior member of the